In [22]:
import os 
import sys 
import json 
import logging 
import builtins 
from typing import List 
from pathlib import Path 
from functools import cache
from itertools import product

# Required when developing in a jupyter-notebook environment 
cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.subgraph import SyntheticField
from subgrounds.pagination import ShallowStrategy

# Required when developing in a jupyter-notebook environment 
load_dotenv('../../../../.env')

# print(os.environ['SUBGRAPH_URL'])

from utils_notebook.utils import ddf, remove_prefix, load_subgraph, remove_keys
from utils_notebook.vega import condition_union, output_chart, apply_css, stack_order_expr, chart, XAXIS_DEFAULTS, possibly_override
from utils_notebook.testing import validate_season_series
from utils_notebook.constants import ADDR_BEANSTALK
from utils_notebook.queries import QueryManager
from utils_notebook.css import css_tooltip_timeseries_multi_colored

In [23]:
sg, bs = load_subgraph()
q = QueryManager(sg, bs) 

In [24]:
@cache
def query_barn(**kwargs): 
    return q.query_barn()

In [25]:
df_barn = query_barn(cache=1)
df_barn = df_barn[['season', 'sprouts', 'sprouts_rinsable']]
df_barn.head()

,season,sprouts,sprouts_rinsable
1,6074,8.643268e+07,0.000000
2,6075,8.644328e+07,0.000000
3,6076,8.654976e+07,2266.788451
4,6077,8.658356e+07,6824.618896
5,6078,8.659188e+07,13760.685470


In [26]:
col_map = {
    'newHarvestablePods': 'pods_harvestable_daily',
    'newHarvestedPods': 'pods_harvested_daily', 
    'podIndex': 'pods_issued_cumulative', 
    'totalHarvestablePods': 'pods_harvestable_cumulative', 
}

In [27]:
@cache
def query_field_daily_snapshots(**kwargs): 
    return q.query_field_daily_snapshots(fields=['season'] + list(col_map.keys())) 

In [28]:
df_field = query_field_daily_snapshots(cache=1).copy()
df_field = df_field.rename(columns=col_map).drop(columns=['timestamp'])
df_field['pods_unharvestable_cumulative'] = df_field.pods_issued_cumulative - df_field.pods_harvestable_cumulative
df_field.tail()

,season,pods_harvestable_daily,pods_harvested_daily,pods_harvestable_cumulative,pods_issued_cumulative,pods_unharvestable_cumulative
332,7953,0.000000,0.00000,5.757145e+07,8.291839e+08,7.716125e+08
333,7977,4913.763716,0.00000,5.757636e+07,8.291875e+08,7.716112e+08
334,8001,5066.463481,11017.37332,5.758143e+07,8.291901e+08,7.716086e+08
335,8025,9452.273142,0.00000,5.759088e+07,8.291943e+08,7.716034e+08
336,8029,2041.022438,0.00000,5.759292e+07,8.291953e+08,7.716024e+08


In [29]:
@cache
def query_silo_daily_snapshots(**kwargs): 
    return q.query_silo_daily_snapshots() 

In [30]:
# process post-replant silo data (subgraph)
df_silo = query_silo_daily_snapshots()
df_silo = df_silo.rename(columns={"dailyBeanMints": "silo_emissions_daily"})
df_silo['silo_emissions_cumulative'] = df_silo.silo_emissions_daily.cumsum()
df_silo.tail()

,season,silo_emissions_daily,silo_emissions_cumulative
2815,7953,0.000000,7.626574e+07
2816,7976,4913.763723,7.627066e+07
2817,8001,5066.463488,7.627572e+07
2818,8025,9452.273156,7.628518e+07
2819,8029,2041.022440,7.628722e+07


In [31]:
@cache 
def query_seasons(**kwargs): 
    return q.query_seasons(extra_cols=['beans'])

In [32]:
df_szns = query_seasons(cache=1)

In [33]:
df = df_szns.merge(
    df_barn, how='left', on='season'
).merge(
    df_field, how='left', on='season'
).merge(
    df_silo, how='left', on='season'
)
df.tail()

,season,timestamp,beans,sprouts,sprouts_rinsable,pods_harvestable_daily,pods_harvested_daily,pods_harvestable_cumulative,pods_issued_cumulative,pods_unharvestable_cumulative,silo_emissions_daily,silo_emissions_cumulative
8025,8025,2022-10-26 23:00:11,3.278625e+07,9.389401e+07,2.137910e+06,9452.273142,0.0,5.759088e+07,8.291943e+08,7.716034e+08,9452.273156,7.628518e+07
8026,8026,2022-10-27 00:00:11,3.278788e+07,9.389351e+07,2.138409e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8027,8027,2022-10-27 01:00:11,3.278950e+07,9.389301e+07,2.138908e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8028,8028,2022-10-27 02:00:11,3.279110e+07,9.389253e+07,2.139390e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8029,8029,2022-10-27 03:00:11,3.279274e+07,9.389203e+07,2.139890e+06,2041.022438,0.0,5.759292e+07,8.291953e+08,7.716024e+08,2041.022440,7.628722e+07


In [34]:

assert len(df) == len(df_szns)
df = df.rename(columns={
    # credit components 
    'sprouts_rinsable': 'fertilized beans', 
    'pods_harvestable_cumulative': 'pods harvestable', 
    'silo_emissions_cumulative': 'silo emissions', 
    # debt components 
    'sprouts': 'unfertilized beans', 
    'pods_unharvestable_cumulative': 'pods unharvestable', 
    # overall 
    'total_debt': 'total debt', 
    'total_credit': 'total credit', 
    'debt_credit_ratio': 'debt credit ratio', 
    'fertilizer_adjusted_pod_rate': 'fertilizer adjusted pod rate', 
    'beans': 'bean_supply',
})
df = df.ffill().fillna(0) # Not technically correct but close enough 
df['total debt'] = (
    df['pods unharvestable'] + df['unfertilized beans']
) 
df['total credit'] = (
    df['fertilized beans'] + df['silo emissions'] + df['pods harvestable']
)
df['debt credit ratio'] = df['total debt'] / df['total credit'] 
df['fertilizer adjusted pod rate'] = df['total debt'] / df['bean_supply'] 
metrics_credit = [
    'silo emissions',
    'pods harvestable',
    'fertilized beans', 
]
metrics_debt = [
    'unfertilized beans', 
    'pods unharvestable', 
]
metrics_credit_debt_aggregate = [
    'total debt', 
    'total credit', 
]
metrics_meta = [
    'debt credit ratio', 
    'fertilizer adjusted pod rate', 
]
metrics = metrics_credit + metrics_debt + metrics_credit_debt_aggregate + metrics_meta
columns = ['timestamp'] + metrics 
df = df[columns]
df = df.resample("W", on="timestamp").last().reset_index()
# df_mask = df['silo emissions'].isna()
timestamp_min = df.timestamp.values[0]
# timestamp_exploit = df[df_mask].timestamp.values[0]
# timestamp_replant = df[df_mask].timestamp.values[-1]
df = df.dropna()
source = df.melt(
    id_vars=['timestamp'], 
    value_vars=metrics, 
).sort_values(["timestamp", "variable"]).reset_index(drop=True)
print(len(source))
source.head(10)

450


,timestamp,variable,value
0,2021-08-08,debt credit ratio,0.067543
1,2021-08-08,fertilized beans,0.000000
2,2021-08-08,fertilizer adjusted pod rate,0.021235
3,2021-08-08,pods harvestable,15863.528585
4,2021-08-08,pods unharvestable,2502.841552
5,2021-08-08,silo emissions,21192.194431
6,2021-08-08,total credit,37055.723016
7,2021-08-08,total debt,2502.841552
8,2021-08-08,unfertilized beans,0.000000
9,2021-08-15,debt credit ratio,0.031606


### Chart Todo's 

- Add pod rate as a metric 


In [35]:
# alt.data_transformers.disable_max_rows()
dropdown = alt.binding_select(
    options=['ymd', 'ym'], labels=["weekly", "monthly"], name='aggregation level:')
selection = alt.selection_single(
    name="agglevel", fields=['AggLevel'], bind=dropdown, init={"AggLevel": 'ymd'}
)
selection_rule = alt.selection_single(
    fields=['tstamp'], nearest=True, on='mouseover', empty='none', clear='mouseout'
)
colors = {
    # credit components 
    'fertilized beans': '#57cc99', # green   
    'pods harvestable': '#38a3a5', # mid blue 
    'silo emissions': '#22577a', # navy blue 
    # debt components 
    'unfertilized beans': "#ef9b20", # Magenta 50
    'pods unharvestable': '#fa4d56', # Red 50
    # overall 
    'total debt': '#9f1853', # Magenta 70 
    'total credit': '#80ed99', # mint green 
    'debt credit ratio': '#ffc300', # gold 
    'fertilizer adjusted pod rate': '#5e60ce' # purple-ish
}
format_decimal = ",d"
format_percent = ".2%"
tooltip_formats = {
    'fertilized beans':  format_decimal,
    'unfertilized beans':  format_decimal,
    'pods harvestable':  format_decimal,
    'silo emissions':  format_decimal,
    'pods unharvestable':  format_decimal,
    'total debt':  format_decimal,
    'total credit':  format_decimal,
    'debt credit ratio': format_percent, 
    'fertilizer adjusted pod rate': format_percent,
}
assert set(colors.keys()) == set(metrics)
assert set(tooltip_formats.keys()) == set(metrics)

base = (
    alt.Chart(source)
    .properties(height=225, width=500)
    .transform_timeunit(
        ymd="yearmonthdate(timestamp)", 
        ym="yearmonth(timestamp)", 
    )
    .transform_calculate(
        tstamp="datum[agglevel.AggLevel]", 
    )
    .transform_aggregate(
        groupby=["tstamp", 'variable'], rvalue='max(value)'
    )
    .transform_calculate(
        # creates numeric stack order key encoding both x position and order of stacked area labels into single value 
        stack_order=f'time(datum.tstamp) + ({stack_order_expr("variable", list(reversed(metrics)))})'
    )
    .encode(
        x=alt.X(
            "tstamp:O", 
            axis=alt.Axis(
                formatType="time", 
                ticks=False, 
                labelExpr="timeFormat(toDate(datum.value), '%b %e, %Y')", 
                labelOverlap=True, 
                labelSeparation=25, 
                labelPadding=5, 
                title="Date", 
                labelAngle=0, 
            ), 
        ),   
    )
)
base_bdv = (
    base
    .encode(
        y=alt.Y("rvalue:Q", axis=alt.Axis(title="BDV", format=".3~s", labelExpr="replace(datum.label, 'G', 'B')")),
        color=alt.Color(
            "variable:N", 
            scale=alt.Scale(
                domain=metrics_credit + metrics_debt + metrics_credit_debt_aggregate, 
                range=[colors[m] for m in metrics_credit + metrics_debt + metrics_credit_debt_aggregate]
            ),
            legend=alt.Legend(title=None)
        ),
        order=alt.Order('stack_order:Q', sort='ascending')
    )
)
base_ratio = base.encode(
    y=alt.Y("rvalue:Q", axis=alt.Axis(title="Percent", format=",%")),
    color=alt.Color(
        "variable:N", 
        scale=alt.Scale(
            domain=metrics_meta, range=[colors[m] for m in metrics_meta]
        ),
        legend=alt.Legend(title=None)
    ),
)
rule_exploit = (
    # selection captures nearest timestamp (for current mouse position) 
    # tooltip rendered uses this data point (pivoted, so we have all data for this timestamp) 
    base
    .transform_pivot('variable', value='rvalue', groupby=['tstamp'])
    .transform_filter("year(datum.tstamp) === 2022 && month(datum.tstamp) === 3 && date(datum.tstamp) === 17")
    .mark_rule(opacity=1, color='#474440', strokeDash=[2.5,1])
)
rule = (
    # selection captures nearest timestamp (for current mouse position) 
    # tooltip rendered uses this data point (pivoted, so we have all data for this timestamp) 
    base
    .transform_pivot('variable', value='rvalue', groupby=['tstamp'])
    .mark_rule(opacity=0)
    .encode(
        tooltip=(
            [alt.Tooltip('tstamp:O', timeUnit="yearmonthdate", title="date")] + 
            [
                alt.Tooltip(
                    f'{m}:Q', 
                    format=tooltip_formats[m], 
                    title=m.replace("_", " ").replace(" cumulative", "")
                ) for m in metrics
            ] 
        ) 
    )
    .add_selection(selection_rule)
)
credit = (
    base_bdv
    .mark_bar()
    .transform_filter(condition_union("==", "|", metrics_credit))
)
debt = (
    base_bdv
    .mark_bar()
    .transform_filter(condition_union("==", "|", metrics_debt))
) 
lines_debt_credit = (
    base_bdv
    .mark_line()
    .transform_filter(condition_union('==', '|', metrics_credit_debt_aggregate))
) 
line_ratio = (
    base_ratio
    .mark_line()
    .transform_filter(condition_union('==', '|', metrics_meta))
)
point_ratio = (
    base_ratio
    .mark_point(size=7)
    .transform_filter(condition_union('==', '|', metrics_meta))
)

c = (
    alt
    .vconcat(
        alt.layer(debt, credit, lines_debt_credit, rule, rule_exploit).properties(title="Beanstalk Credit Profile"),
        alt.layer(line_ratio, point_ratio, rule, rule_exploit).properties(title="Beanstalk Credit Metrics"),
    )
    .resolve_legend(color="independent")
    .resolve_axis(y="independent")
    .resolve_scale(y="independent", color="independent")
    .add_selection(selection) 
)


css_lines = [
    "div.chart-wrapper { display: flex; flex-direction: column; }", 
    "form.vega-bindings { display: block; order: -1; }", 
    "canvas { order: 1 }", 
    """
    div.vega-bind { 
        display: inline-block; 
        padding: 5px; 
    }
    span.vega-bind-name { 
        font-weight: 500 !important; 
        padding-right: 5px !important; 
    }
    span.vega-bind-name span { 
        font-weight: 600 !important; 
        padding-right: 5px !important; 
    }
    div.vega-bind select { 
        border: .5px solid #000000;
        border-radius: 3px;
    }
    """,
]
css_lines = css_lines + css_tooltip_timeseries_multi_colored(metrics, colors) 
css = "\n".join(css_lines)
apply_css("")
# apply_css(css) 
c

/Users/ALEX/opt/anaconda3/envs/beanstalk/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

In [148]:
def chart(
    df: pd.DataFrame, 
    timestamp_col: str, 
    lmetrics: List[str], 
    rmetrics: List[str] = None, 
    lstrategy: str = 'line', 
    rstrategy: str = 'line', 
    title: str = '', 
    xaxis_kwargs = None, 
    xaxis_kwargs_override: bool = False, 
    yaxis_left_kwargs: dict = None, 
    yaxis_left_kwargs_override: bool = False, 
    yaxis_right_kwargs: dict = None, 
    yaxis_right_kwargs_override: bool = False, 
    color_map = None,      
    tooltip_formats = None, 
    tooltip_metrics = None, 
    dual_axes: bool = False, 
    show_exploit_rule: bool = True, 
    exploit_day: int = 17, # must be either 16 or 17
    width: int = 700, 
    selection_nearest: alt.selection = None, 
    return_selection: bool = False, 
    base_hook = None, 
): 
    """Creates a chart with a shared time axis and up to two y axes 
        
    Assumes that data is in long-wide format (i.e. df was processed with function wide_to_longwide)
    """
    rmetrics = rmetrics or []
    assert not set(lmetrics).intersection(set(rmetrics)), "Same metric on two axes"
    metrics = lmetrics + rmetrics
    tooltip_formats = tooltip_formats or {}
    xaxis_kwargs = possibly_override(xaxis_kwargs, XAXIS_DEFAULTS, override=xaxis_kwargs_override)
    yaxis_left_kwargs = possibly_override(yaxis_left_kwargs, None, override=yaxis_left_kwargs_override)
    yaxis_right_kwargs = possibly_override(yaxis_right_kwargs, None, override=yaxis_right_kwargs_override)

    # Selection for nearest point 
    if not selection_nearest: 
        selection_nearest = alt.selection_single(
            fields=[timestamp_col], nearest=True, on='mouseover', empty='none', clear='mouseout'
        )

    # Color Scale 
    if color_map: 
        color_scale = alt.Scale(domain=metrics, range=[color_map[m] for m in metrics])
    else: 
        color_scale = alt.Scale(domain=metrics)
    
    base = (
        alt.Chart(df)
        .transform_calculate(stack_order=stack_order_expr("variable", metrics))
        .encode(x=alt.X(f"{timestamp_col}:O", axis=alt.Axis(**xaxis_kwargs)))
        .properties(title=title, width=width)
    )
    if base_hook: 
        base = base_hook(base)
        
    cbase = (
        base
        
        .encode(
            color=alt.Color("variable:N", scale=color_scale, legend=alt.Legend(title=None)), 
            order=alt.Order('stack_order:Q', sort='ascending'),
        )
    )

    class Strategies: 

        @staticmethod
        def line(base, axis):
            return (
                base 
                .mark_line()
                .encode(y=alt.Y("value:Q", axis=axis))
            )
        
        @staticmethod
        def point(base, axis):
            return (
                base 
                .mark_point()
                .encode(y=alt.Y("value:Q", axis=axis))
            )
        
        @staticmethod
        def stack_area(base, axis):
            return (
                base 
                .transform_calculate(sort_col=stack_order_expr("variable", metrics))
                .mark_area(point='transparent')
                .encode(y=alt.Y("value:Q", axis=axis)) 
            )
            
        @staticmethod
        def stack_bar(base, axis):
            return (
                base 
                .transform_calculate(sort_col=stack_order_expr("variable", metrics))
                .mark_bar()
                .encode(y=alt.Y("value:Q", axis=axis)) 
            )

    strategies = {
        "line": Strategies.line, 
        "point": Strategies.point, 
        "stack_area": Strategies.stack_area, 
        "stack_bar": Strategies.stack_bar,
    }
    
    left_wrapper = dict(chart=None)
    right_wrapper = dict(chart=None)
    chart_specs = [
        (lstrategy, lmetrics, yaxis_left_kwargs, left_wrapper), 
    ]
    if rmetrics: 
        chart_specs.append((rstrategy, rmetrics, yaxis_right_kwargs, right_wrapper))
    
    for strategy, smetrics, axis_kwargs, chart_wrapper in chart_specs:
        match type(strategy): 
            case builtins.str: 
                # Apply a single strategy to all metrics on this axis 
                chart_wrapper['chart'] = strategies[strategy](
                    cbase.transform_filter(condition_union("==", "|", smetrics)),
                    alt.Axis(**axis_kwargs)
                ) 
            case builtins.list: 
                # Apply strategies on a per-metric basis 
                assert len(strategy) == len(smetrics)
                df_strategy_metric = pd.DataFrame(dict(strategy=strategy, metrics=smetrics))
                layers = []
                order = {
                    "stack_area": 0, 
                    "stack_bar": 1,
                    "line": 2, 
                    "point": 3, 
                }
                ax = alt.Axis(**axis_kwargs)
                for strategy, df_sm in sorted(
                    df_strategy_metric.groupby("strategy"), key=lambda e: order[e[0]]
                ): 
                    sub_metrics = df_sm.metrics.values.tolist()
                    layer = strategies[strategy](
                        cbase.transform_filter(condition_union("==", "|", sub_metrics)), ax
                    ) 
                    layers.append(layer)
                chart_wrapper['chart'] = alt.layer(*layers)  
            case _: 
                raise ValueError(f"Invalid strategy {strategy}")
            
    left = left_wrapper['chart']
    right = right_wrapper['chart']

    tooltip_metrics = tooltip_metrics or metrics 
    nearest = (
        # selection captures nearest timestamp (for current mouse position) 
        # tooltip rendered uses this data point (pivoted, so we have all data for this timestamp) 
        base
        .transform_pivot('variable', value='value', groupby=[timestamp_col])
        .mark_rule(color="#878787")
        .encode(
            tooltip=(
                [alt.Tooltip(f'{timestamp_col}:O', timeUnit="yearmonthdate", title="date")] + 
                [alt.Tooltip(f'{m}:Q', format=tooltip_formats.get(m, ",d")) for m in tooltip_metrics]
            ), 
            opacity=alt.condition(selection_nearest, alt.value(1), alt.value(0))
        )
        .add_selection(selection_nearest)
    )

    assert exploit_day in [16, 17]
    rule_exploit = (
        # selection captures nearest timestamp (for current mouse position) 
        # tooltip rendered uses this data point (pivoted, so we have all data for this timestamp) 
        base
        .transform_pivot('variable', value='value', groupby=[timestamp_col])
        .transform_filter(f"""
            year(datum['{timestamp_col}']) === 2022 && 
            month(datum['{timestamp_col}']) === 3 && 
            date(datum['{timestamp_col}']) === {exploit_day} 
        """) # && warn(datetime(datum['{timestamp_col}']))
        .mark_rule(opacity=1, color='#474440', strokeDash=[2.5,1])
    )

    # Compose plot 
    if not rmetrics: 
        if show_exploit_rule: 
            c = left + rule_exploit + nearest
        else: 
            c = left + nearest
    else: 
        if show_exploit_rule: 
            # It matters that the rules are layered with right instead of left, not sure why. 
            # Parentheses are important in case where dual_axes is True 
            c = left + (right + rule_exploit + nearest)
        else: 
            # It matters that the rules are layered with right instead of left, not sure why. 
            # Parentheses are important in case where dual_axes is True 
            c = left + (right + nearest)
    if dual_axes: 
        assert rmetrics, "Can't have two axes if you didn't specify rmetrics" 
        c = (
            c
            .resolve_scale(y="independent")
            .resolve_axis(y="independent")
        )
    return c if not return_selection else (c, selection_nearest)

In [149]:
source.head()

,timestamp,variable,value
0,2021-08-08,debt credit ratio,0.067543
1,2021-08-08,fertilized beans,0.000000
2,2021-08-08,fertilizer adjusted pod rate,0.021235
3,2021-08-08,pods harvestable,15863.528585
4,2021-08-08,pods unharvestable,2502.841552


In [150]:
# alt.data_transformers.disable_max_rows()
dropdown = alt.binding_select(
    options=['ymd', 'ym'], 
    labels=["weekly", "monthly"], 
    name='aggregation level:'
)
selection = alt.selection_single(
    name="agglevel", 
    fields=['AggLevel'], 
    bind=dropdown, 
    init={"AggLevel": 'ymd'}
)
colors = {
    # credit components 
    'fertilized beans': '#57cc99', # green   
    'pods harvestable': '#38a3a5', # mid blue 
    'silo emissions': '#22577a', # navy blue 
    # debt components 
    'unfertilized beans': "#ef9b20", # Magenta 50
    'pods unharvestable': '#fa4d56', # Red 50
    # overall 
    'total debt': '#9f1853', # Magenta 70 
    'total credit': '#80ed99', # mint green 
    'debt credit ratio': '#ffc300', # gold 
    'fertilizer adjusted pod rate': '#5e60ce' # purple-ish
}
format_decimal = ",d"
format_percent = ".2%"
tooltip_formats = {
    'fertilized beans':  format_decimal,
    'unfertilized beans':  format_decimal,
    'pods harvestable':  format_decimal,
    'silo emissions':  format_decimal,
    'pods unharvestable':  format_decimal,
    'total debt':  format_decimal,
    'total credit':  format_decimal,
    'debt credit ratio': format_percent, 
    'fertilizer adjusted pod rate': format_percent,
}
assert set(colors.keys()) == set(metrics)
assert set(tooltip_formats.keys()) == set(metrics)

def base_hook(c): 
    return (
        c
        .transform_timeunit(
            ymd="yearmonthdate(timestamp)", 
            ym="yearmonth(timestamp)", 
        )
        .transform_calculate(
            tstamp="datum[agglevel.AggLevel]", 
        )
        .transform_aggregate(
            groupby=["tstamp", 'variable'], value='max(value)'
        )
    )

tooltip_metrics = metrics_debt + metrics_credit + ['total debt']
(
    alt.vconcat(
        (
            alt.layer(
                chart(
                    source, 
                    "tstamp", 
                    lmetrics=list(reversed(metrics_debt)) + ['total debt'], 
                    lstrategy=["stack_bar" for i in range(len(metrics_debt))] + ['line'], 
                    tooltip_metrics=tooltip_metrics, 
                    color_map=colors,
                    base_hook=base_hook
                ), 
                chart(
                    source, 
                    "tstamp", 
                    lmetrics=['total credit'] + metrics_credit, 
                    lstrategy=['line'] + ["stack_bar" for i in range(len(metrics_credit))], 
                    tooltip_metrics=tooltip_metrics, 
                    color_map=colors, 
                    base_hook=base_hook
                )
            )
            .resolve_legend(color="independent")
            .resolve_scale(color="independent")
        ),
        alt.layer(
            chart(
                source, 
                "tstamp", 
                lmetrics=metrics_meta, 
                lstrategy="line", 
                color_map=colors, 
                base_hook=base_hook
            ),
            chart(
                source, 
                "tstamp", 
                lmetrics=metrics_meta, 
                lstrategy="point", 
                color_map=colors, 
                base_hook=base_hook
            ),
        )
    )
    .add_selection(selection)
)

# base = (
#     alt.Chart(source)
#     .properties(height=225, width=500)
#     .encode(
#         x=alt.X(
#             "tstamp:O", 
#             axis=alt.Axis(
#                 formatType="time", 
#                 ticks=False, 
#                 labelExpr="timeFormat(toDate(datum.value), '%b %e, %Y')", 
#                 labelOverlap=True, 
#                 labelSeparation=25, 
#                 labelPadding=5, 
#                 title="Date", 
#                 labelAngle=0, 
#             ), 
#         ),   
#     )
# )
# base_bdv = (
#     base
#     .encode(
#         y=alt.Y("rvalue:Q", axis=alt.Axis(title="BDV", format=".3~s", labelExpr="replace(datum.label, 'G', 'B')")),
#         color=alt.Color(
#             "variable:N", 
#             scale=alt.Scale(
#                 domain=metrics_credit + metrics_debt + metrics_credit_debt_aggregate, 
#                 range=[colors[m] for m in metrics_credit + metrics_debt + metrics_credit_debt_aggregate]
#             ),
#             legend=alt.Legend(title=None)
#         ),
#         order=alt.Order('stack_order:Q', sort='ascending')
#     )
# )
# base_ratio = base.encode(
#     y=alt.Y("rvalue:Q", axis=alt.Axis(title="Percent", format=",%")),
#     color=alt.Color(
#         "variable:N", 
#         scale=alt.Scale(
#             domain=metrics_meta, range=[colors[m] for m in metrics_meta]
#         ),
#         legend=alt.Legend(title=None)
#     ),
# )
# rule_exploit = (
#     # selection captures nearest timestamp (for current mouse position) 
#     # tooltip rendered uses this data point (pivoted, so we have all data for this timestamp) 
#     base
#     .transform_pivot('variable', value='rvalue', groupby=['tstamp'])
#     .transform_filter("year(datum.tstamp) === 2022 && month(datum.tstamp) === 3 && date(datum.tstamp) === 17")
#     .mark_rule(opacity=1, color='#474440', strokeDash=[2.5,1])
# )
# rule = (
#     # selection captures nearest timestamp (for current mouse position) 
#     # tooltip rendered uses this data point (pivoted, so we have all data for this timestamp) 
#     base
#     .transform_pivot('variable', value='rvalue', groupby=['tstamp'])
#     .mark_rule(opacity=0)
#     .encode(
#         tooltip=(
#             [alt.Tooltip('tstamp:O', timeUnit="yearmonthdate", title="date")] + 
#             [
#                 alt.Tooltip(
#                     f'{m}:Q', 
#                     format=tooltip_formats[m], 
#                     title=m.replace("_", " ").replace(" cumulative", "")
#                 ) for m in metrics
#             ] 
#         ) 
#     )
#     .add_selection(selection_rule)
# )
# credit = (
#     base_bdv
#     .mark_bar()
#     .transform_filter(condition_union("==", "|", metrics_credit))
# )
# debt = (
#     base_bdv
#     .mark_bar()
#     .transform_filter(condition_union("==", "|", metrics_debt))
# ) 
# lines_debt_credit = (
#     base_bdv
#     .mark_line()
#     .transform_filter(condition_union('==', '|', metrics_credit_debt_aggregate))
# ) 
# line_ratio = (
#     base_ratio
#     .mark_line()
#     .transform_filter(condition_union('==', '|', metrics_meta))
# )
# point_ratio = (
#     base_ratio
#     .mark_point(size=7)
#     .transform_filter(condition_union('==', '|', metrics_meta))
# )

# c = (
#     alt
#     .vconcat(
#         alt.layer(debt, credit, lines_debt_credit, rule, rule_exploit).properties(title="Beanstalk Credit Profile"),
#         alt.layer(line_ratio, point_ratio, rule, rule_exploit).properties(title="Beanstalk Credit Metrics"),
#     )
#     .resolve_legend(color="independent")
#     .resolve_axis(y="independent")
#     .resolve_scale(y="independent", color="independent")
#     .add_selection(selection) 
# )


# css_lines = [
#     "div.chart-wrapper { display: flex; flex-direction: column; }", 
#     "form.vega-bindings { display: block; order: -1; }", 
#     "canvas { order: 1 }", 
#     """
#     div.vega-bind { 
#         display: inline-block; 
#         padding: 5px; 
#     }
#     span.vega-bind-name { 
#         font-weight: 500 !important; 
#         padding-right: 5px !important; 
#     }
#     span.vega-bind-name span { 
#         font-weight: 600 !important; 
#         padding-right: 5px !important; 
#     }
#     div.vega-bind select { 
#         border: .5px solid #000000;
#         border-radius: 3px;
#     }
#     """,
# ]
# css_lines = css_lines + css_tooltip_timeseries_multi_colored(metrics, colors) 
# css = "\n".join(css_lines)
# apply_css("")
# # apply_css(css) 
# c

/Users/ALEX/opt/anaconda3/envs/beanstalk/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)